In [2]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import difflib
from pulp import *
import openpyxl
import glob

from datetime import *
today = date.today()
today = today.strftime("%m.%d.20%y")
today

# import datetime
# today = date.today()
# today = today - datetime.timedelta(days=1)
# today = today.strftime("%m.%d.20%y")
# today

'03.13.2022'

In [49]:
## Uploads
os.chdir(r"C:\Users\vchang\Downloads")

fanduel_upload = r"C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\FanDuel"

fd_upload = glob.glob('*upload-template.csv')[0]
os.rename(fd_upload, "FD_Upload_"+ contest + str(today)+".csv")

shutil.move(downloads + "/"+ "FD_Upload_"+ contest + str(today)+".csv",
           fanduel_upload + "/"+ "FD_Upload_"+ contest + str(today)+".csv")


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'FanDuel-NBA-2022-03-11-72786-entries-upload-template.csv' -> 'FD_Upload_PlayerPool_03.11.2022.csv'

In [50]:
## FanDuel Player Pool

contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool'

fd = glob.glob('*-players-list.csv')[0]
os.rename(fd, "FDSalaries_"+ contest + str(today)+".csv")

shutil.move(downloads + "/"+ "FDSalaries_"+ contest + str(today)+".csv",
           contest_data_loc + "/"+ "FDSalaries_"+ contest + str(today)+".csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool")
fanduel_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + str(today)+".csv")
fanduel_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + str(today)+".xlsx", index=False)
os.remove("FDSalaries_"+ contest + str(today)+".csv")


In [51]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/FanDuel'
daily_file = "FD NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+str(today)+".xlsx", index=False)
os.remove("FD NBA Projections.csv")

In [52]:
## Joins the FD and ETR Files to include FD Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\FanDuel\ETR_Daily_' + str(today) + ".xlsx")
fd = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\FD_Data\PlayerPool\FDSalaries_PlayerPool_' + str(today) + ".xlsx")

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              fd[['Nickname','Position', 'Id']], 
              left_on='Player', 
              right_on='Nickname')

In [53]:
df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id
0,LeBron James,LAL,WAS,37.5,SF/PF,"10,900",55.8,7.2,65.9,47.6,MAIN,LeBron James,SF/PF,72788-9488
1,Kristaps Porzingis,WAS,LAL,27.0,PF/C,"6,000",35.2,6.2,43.7,33.8,MAIN,Kristaps Porzingis,PF/C,72788-63122
2,Russell Westbrook,LAL,WAS,33.5,PG,"7,600",38.9,3.5,48.4,7.6,MAIN,Russell Westbrook,PG,72788-9642
3,Pascal Siakam,TOR,PHX,38.0,PF,"8,100",40.8,3.4,49.6,12.8,MAIN,Pascal Siakam,PF,72788-60021
4,Cameron Payne,PHX,TOR,33.0,PG/SG,"6,700",34.5,2.7,43.4,13.8,MAIN,Cameron Payne,PG/SG,72788-48567


In [54]:
# Add 'G' Column
# Add 'F' Column
df['G Position'] = np.where(df['FD Position'].str.contains("G"), 'G', "")
df['F Position'] = np.where(df['FD Position'].str.contains("F"), 'F', "")

df['G Position'] = df['G Position'].replace('', np.nan, regex=True)
df['F Position'] = df['F Position'].replace('', np.nan, regex=True)

df['list'] = df[['FD Position', 'G Position', 'F Position']].apply(lambda x: '/'.join(x.dropna()), axis=1)

# ## Update Salary to INT 
df['FD Salary'] = df['FD Salary'].str.replace(",","").astype(float)

df['Name + ID'] = df['Id'] + ":" + df['Player']

df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,G Position,F Position,list,Name + ID
0,LeBron James,LAL,WAS,37.5,SF/PF,10900.0,55.8,7.2,65.9,47.6,MAIN,LeBron James,SF/PF,72788-9488,NaN,F,SF/PF/F,72788-9488:LeBron James
1,Kristaps Porzingis,WAS,LAL,27.0,PF/C,6000.0,35.2,6.2,43.7,33.8,MAIN,Kristaps Porzingis,PF/C,72788-63122,NaN,F,PF/C/F,72788-63122:Kristaps Porzingis
2,Russell Westbrook,LAL,WAS,33.5,PG,7600.0,38.9,3.5,48.4,7.6,MAIN,Russell Westbrook,PG,72788-9642,G,NaN,PG/G,72788-9642:Russell Westbrook
3,Pascal Siakam,TOR,PHX,38.0,PF,8100.0,40.8,3.4,49.6,12.8,MAIN,Pascal Siakam,PF,72788-60021,NaN,F,PF/F,72788-60021:Pascal Siakam
4,Cameron Payne,PHX,TOR,33.0,PG/SG,6700.0,34.5,2.7,43.4,13.8,MAIN,Cameron Payne,PG/SG,72788-48567,G,NaN,PG/SG/G,72788-48567:Cameron Payne


In [55]:
df.to_csv("FD_final_PlayerPool_ETR_"+str(today)+".csv", index=False)

df = pd.read_csv(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis\FD_final_PlayerPool_ETR_'+ str(today) + ".csv")

df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,G Position,F Position,list,Name + ID
0,LeBron James,LAL,WAS,37.5,SF/PF,10900.0,55.8,7.2,65.9,47.6,MAIN,LeBron James,SF/PF,72788-9488,NaN,F,SF/PF/F,72788-9488:LeBron James
1,Kristaps Porzingis,WAS,LAL,27.0,PF/C,6000.0,35.2,6.2,43.7,33.8,MAIN,Kristaps Porzingis,PF/C,72788-63122,NaN,F,PF/C/F,72788-63122:Kristaps Porzingis
2,Russell Westbrook,LAL,WAS,33.5,PG,7600.0,38.9,3.5,48.4,7.6,MAIN,Russell Westbrook,PG,72788-9642,G,NaN,PG/G,72788-9642:Russell Westbrook
3,Pascal Siakam,TOR,PHX,38.0,PF,8100.0,40.8,3.4,49.6,12.8,MAIN,Pascal Siakam,PF,72788-60021,NaN,F,PF/F,72788-60021:Pascal Siakam
4,Cameron Payne,PHX,TOR,33.0,PG/SG,6700.0,34.5,2.7,43.4,13.8,MAIN,Cameron Payne,PG/SG,72788-48567,G,NaN,PG/SG/G,72788-48567:Cameron Payne


# Create the Constraint Problem

Goal: Maximize FD Points

- Total Players = 9
- TotalSalary <= 60000
- TotalPosition_PG = 2
- TotalPosition_SG = 2
- TotalPosition_SF = 2
- TotalPosition_PF =2 
- TotalPosition_C =1


In [56]:
## Unnests FD Positions into each Row (explode in Python) ##


## Update Salary to INT 
#df['DK Salary'] = df['DK Salary'].str.replace(",","").astype(float)

df['FD Position'] = df['FD Position'].astype(str)
df['FD Position'] = df['FD Position'].str.split('/')

df = df.explode('FD Position')

df['Position + Id'] = df['FD Position'] + "_" + df['Id']

df.head()

df.to_csv("FD_final_PlayerPool_ETR_"+str(today)+".csv", index=False)
# os.system("DKNG_final_PlayerPool_ETR_"+str(today)+".xlsx")

In [57]:
df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,G Position,F Position,list,Name + ID,Position + Id
0,LeBron James,LAL,WAS,37.5,SF,10900.0,55.8,7.2,65.9,47.6,MAIN,LeBron James,SF/PF,72788-9488,NaN,F,SF/PF/F,72788-9488:LeBron James,SF_72788-9488
0,LeBron James,LAL,WAS,37.5,PF,10900.0,55.8,7.2,65.9,47.6,MAIN,LeBron James,SF/PF,72788-9488,NaN,F,SF/PF/F,72788-9488:LeBron James,PF_72788-9488
1,Kristaps Porzingis,WAS,LAL,27.0,PF,6000.0,35.2,6.2,43.7,33.8,MAIN,Kristaps Porzingis,PF/C,72788-63122,NaN,F,PF/C/F,72788-63122:Kristaps Porzingis,PF_72788-63122
1,Kristaps Porzingis,WAS,LAL,27.0,C,6000.0,35.2,6.2,43.7,33.8,MAIN,Kristaps Porzingis,PF/C,72788-63122,NaN,F,PF/C/F,72788-63122:Kristaps Porzingis,C_72788-63122
2,Russell Westbrook,LAL,WAS,33.5,PG,7600.0,38.9,3.5,48.4,7.6,MAIN,Russell Westbrook,PG,72788-9642,G,NaN,PG/G,72788-9642:Russell Westbrook,PG_72788-9642


In [58]:
df = pd.read_csv("FD_final_PlayerPool_ETR_"+str(today)+".csv", index_col=['Id', 'FD Position'], skipinitialspace=True)

legal_assignments = df.index # tuples of (name, pos)
name_set = df.index.unique(0) # a convenience

costs = df['FD Salary'].to_dict()
values = df['FD Points'].to_dict()

# set up LP
draft = pulp.LpVariable.dicts('selected', legal_assignments, cat='Binary')

prob = pulp.LpProblem('the draft', LpMaximize)

#obj
prob += pulp.lpSum([draft[n,p]*values[n,p] for (n,p) in legal_assignments])

#salary cap
prob += pulp.lpSum([draft[n,p]*costs[n,p] for (n, p) in legal_assignments]) <= 60000

# positions
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'PG']) == 2
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'SG']) == 2
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'SF']) == 2
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'PF']) == 2
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'C']) == 1

# use each player at most only once
result = []
for name in name_set:
    prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if n == name]) <= 1

prob.solve()

for idx in draft:
    if draft[idx].varValue > 0:
        print(idx[0], idx[1])
        
        result.append([idx[0], idx[1]])

72788-9488 SF
72788-63122 PF
72788-9642 PG
72788-48567 SG
72788-24247 PG
72788-84670 SF
72788-68203 PF
72788-9739 C
72788-15636 SG


C:\Users\vchang\Anaconda3\lib\site-packages\pulp\pulp.py:1313: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [59]:
# new_list = result
# df = pd.DataFrame(new_list)
# test = df[[1,0]]
# test1 = test.transpose()
# test2 = test1.rename(columns=test1.iloc[0])
# test3 = test2.drop([1])
# df2 = test3[['PG', 'SG','SF','PF', 'C']]
# df2.to_csv("fd_result_" + str(today) + ".csv", index=None)

In [60]:
new_list = result
df = pd.DataFrame(new_list)
test = df[[1,0]]

# Create a lookup variable for [Position + Id]
test['lookup'] = test[1] + "_" + test[0]


# Merge PuLP output on [Position + Id] to pull in Player Name
df = pd.read_csv("FD_final_PlayerPool_ETR_"+str(today)+".csv")
upload = pd.merge(test,
                 df[['Player', 'Position + Id', 'Name + ID']],
                 left_on = 'lookup',
                 right_on = 'Position + Id')


# Drop all columns not used in upload
upload2 = upload.drop([0, 'lookup', 'Player', 'Position + Id'], axis = 1)


# Transpose data output from vertical to horizontal
upload3 = upload2.transpose()


# Rename columns to first row (position values)
upload4 = upload3.rename(columns=upload3.iloc[0])


# Drop duplicate first row
upload5 = upload4.drop([1])


# Re-order the columns to the Spositions
upload6 = upload5[['PG','SG','SF','PF','C']]


# Save as a csv
upload6.to_csv("fd_result_" + str(today) + ".csv", index=None)

os.system("fd_result_" + str(today) + ".csv")

upload6

C:\Users\vchang\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,PG,PG,SG,SG,SF,SF,PF,PF,C
Name + ID,72788-9642:Russell Westbrook,72788-24247:Fred VanVleet,72788-48567:Cameron Payne,72788-15636:Kentavious Caldwell-Pope,72788-9488:LeBron James,72788-84670:Mikal Bridges,72788-63122:Kristaps Porzingis,72788-68203:Chris Boucher,72788-9739:JaVale McGee


In [32]:
upload6

,PG,PG.1,SG,SG.1,SF,SF.1,PF,PF.1,C
0,72788-9642:Russell Westbrook,72788-24247:Fred VanVleet,72788-48567:Cameron Payne,72788-15636:Kentavious Caldwell-Pope,72788-9488:LeBron James,72788-84670:Mikal Bridges,72788-63122:Kristaps Porzingis,72788-68203:Chris Boucher,72788-9739:JaVale McGee


In [33]:
upload6 = pd.read_csv(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis\fd_result_' + str(today) + ".csv")
upload6.head()

,PG,PG.1,SG,SG.1,SF,SF.1,PF,PF.1,C
0,72788-9642:Russell Westbrook,72788-24247:Fred VanVleet,72788-48567:Cameron Payne,72788-15636:Kentavious Caldwell-Pope,72788-9488:LeBron James,72788-84670:Mikal Bridges,72788-63122:Kristaps Porzingis,72788-68203:Chris Boucher,72788-9739:JaVale McGee


In [34]:
upload = pd.read_csv(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\FanDuel\FD_Upload_PlayerPool_' + str(today) + ".csv")
upload = upload[['entry_id', 'contest_id', 'contest_name']]
upload.head()

,entry_id,contest_id,contest_name
0,2.788884e+09,72788-255606169,$5K Fri NBA Block (Single Entry)
1,2.788884e+09,72788-255618736,NBA Mini Swat (EPE)
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN


In [35]:
os.chdir(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\FanDuel')
result = pd.concat([upload, upload6], axis=1)
test = result.rename(columns={"PG.1":"PG", "SG.1":"SG", "SF.1":"SF", "PF.1":"PF"})
result.to_csv("test_"+str(today)+".csv", index=False)

,entry_id,contest_id,contest_name,PG,PG,SG,SG,SF,SF,PF,PF,C
0,2.788884e+09,72788-255606169,$5K Fri NBA Block (Single Entry),72788-9642:Russell Westbrook,72788-24247:Fred VanVleet,72788-48567:Cameron Payne,72788-15636:Kentavious Caldwell-Pope,72788-9488:LeBron James,72788-84670:Mikal Bridges,72788-63122:Kristaps Porzingis,72788-68203:Chris Boucher,72788-9739:JaVale McGee
1,2.788884e+09,72788-255618736,NBA Mini Swat (EPE),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
